In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset_movies = pd.read_csv('movies (1).csv')
dataset_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:

dataset_ratings = pd.read_csv('ratings (1).csv')
dataset_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
n_userID=len(dataset_ratings['userId'].unique())
n_movieID=len(dataset_ratings['movieId'].unique())
n_rating=len(dataset_ratings)
print("Number of Ratings: ",n_rating)
print("Number of user IDs: ",n_userID)
print('Number of Movie IDs: ', n_movieID)
print('Average Ratings per User ID:', round(n_rating/n_userID))
print('Average Ratings per Movie ID:',round(n_rating/n_movieID))

Number of Ratings:  100836
Number of user IDs:  610
Number of Movie IDs:  9724
Average Ratings per User ID: 165
Average Ratings per Movie ID: 10


In [5]:
n_user_rating=dataset_ratings[['userId','rating']].groupby('userId').count().reset_index()
n_user_rating.head()

,userId,rating
0,1,232
1,2,29
2,3,39
3,4,216
4,5,44


In [6]:
mean_rating=dataset_ratings.groupby('movieId')[['rating']].mean()
lowest_rating=mean_rating['rating'].idxmin()
print('Number of Low Rating Movies:',lowest_rating)
highest_rating=mean_rating['rating'].idxmax()
print('Number of High Rating Movies:',highest_rating)
movie_stats = dataset_ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()
movie_stats.head()


Number of Low Rating Movies: 3604
Number of High Rating Movies: 53


,count,mean
movieId,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429


In [7]:
# Add this code before running your main code to debug:
print("Available columns:", dataset_ratings.columns.tolist())
print("\nFirst few rows of your data:")
print(dataset_ratings.head())

Available columns: ['userId', 'movieId', 'rating', 'timestamp']

First few rows of your data:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


In [8]:
from scipy.sparse import csr_matrix
def Matrix(df):
    n=len(df['userId'].unique())
    m=len(df['movieId'].unique())
    #Ids to indexes
    user_map_to_index=dict(zip(np.unique(df['userId']), list(range(n))))
    movie_map_to_index=dict(zip(np.unique(df['movieId']), list(range(m))))
    #Indexes to IDs
    user_inv=dict(zip(np.unique(list(range(n))), df['userId']))
    movie_inv=dict(zip(np.unique(list(range(m))), df['movieId']))
    user_index=[user_map_to_index[i] for i in df['userId']]
    movie_index=[movie_map_to_index[i] for i in df['movieId']]
    X = csr_matrix((df['rating'], (movie_index,user_index)), shape=(m,n))
    return X,user_map_to_index,movie_map_to_index,user_inv,movie_inv
X,user_map_to_index,movie_map_to_index,user_inv,movie_inv=Matrix(dataset_ratings)

    

In [9]:
from sklearn.neighbors import NearestNeighbors
def find_similar_movies(movie_id,X,k,metric='cosine',show_distance=False):
    neighbor_id=[]
    movie_ind=movie_map_to_index[movie_id]
    movie_vec=X[movie_ind]
    k=k+1
    knn=NearestNeighbors(n_neighbors=k,algorithm='brute',metric=metric)
    knn.fit(X)
    movie_vec= movie_vec.reshape(1,-1)
    neighbour=knn.kneighbors(movie_vec,return_distance=show_distance)
    for i in range(0,k):
        n=neighbour.item(i)
        neighbor_id.append(movie_inv[n])
    neighbor_id.pop(0)
    return neighbor_id
movie_titles=dict(zip(dataset_movies['movieId'], dataset_movies['title']))
movie_id=int(input(print("enter movie id")))
similar_ids = find_similar_movies(movie_id,X,k=10)
movie_title=movie_titles[movie_id]
print("Since you watched",movie_title)
for i in similar_ids:
    print(movie_titles[i])

enter movie id
Since you watched Grumpier Old Men (1995)
Bean (1997)
Happy Gilmore (1996)
Clockers (1995)
Bio-Dome (1996)
Usual Suspects, The (1995)
Groundhog Day (1993)
Ghost and the Darkness, The (1996)
Leaving Las Vegas (1995)
Bottle Rocket (1996)
Labyrinth (1986)


In [10]:
def recommend_movies(user_id,X,user_map_to_index,movie_map_to_index,movie_inv,k=10):
    df1=dataset_ratings[dataset_ratings['userId']==user_id]
    if df1.empty:
        print("User with ID ",user_id," does not exist")
        return
    movie_id=df1[df1['rating']==max(df1['rating'])]['movieId'].iloc[0]
    movie_titles=dict(zip(dataset_movies['movieId'], dataset_movies['title']))
    similar_ids=find_similar_movies(movie_id,X,k)
    movie_title=movie_titles.get(movie_id,"Movie not found")
    if movie_title =="Movie not found":
        print("Movie with ID",movie_id,"not found")
        return
    print("Since you watched",movie_title,"you might also like: ")
    for i in similar_ids:
        print(movie_titles.get(i,"Movie nor found"))


In [11]:
user_id=int(input("Enter user id: "))
recommend_movies(user_id,X,user_map_to_index,movie_map_to_index,movie_inv,k=10)

Since you watched Seven (a.k.a. Se7en) (1995) you might also like: 
The Drop (2014)
Together (Tillsammans) (2000)
Grosse Pointe Blank (1997)
Angus (1995)
Circle of Friends (1995)
Adventures of Robin Hood, The (1938)
Whiplash (2014)
Lady and the Tramp (1955)
Wolf of Wall Street, The (2013)
There's Something About Mary (1998)
